## Testing Banding noise vs rotator angle

Craig Lage - 11 Nov 22


In [ ]:
import asyncio
import time
import os
import numpy as np
from astropy.time import Time

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType

In [ ]:
logger = logging.getLogger(f"Daytime Checkout {Time.now()} UT")
logger.level = logging.DEBUG

In [ ]:
logger.info(os.environ["OSPL_URI"])
logger.info(os.environ["LSST_DDS_PARTITION_PREFIX"])

### Instantiate the control classes

In [ ]:
domain = salobj.Domain()
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

In [ ]:
#await atcs.enable()
await latiss.enable()

In [ ]:
await atcs.disable_dome_following()

In [ ]:
current_az = 0.0
current_el = 80.0
current_rot = -160.0
await atcs.point_azel(current_az, current_el, rot_tel=current_rot)

In [ ]:
await latiss.take_bias(nbias=1)

In [ ]:
# Take 2 biases to get things cleared
for i in range(2):
    await asyncio.sleep(2.0)
    await latiss.take_bias(nbias=1)


In [ ]:
# Take biases every 2 degrees for banding tests
for i in range(160):
    current_rot += 2.0
    await latiss.take_bias(nbias=1)
    await atcs.point_azel(current_az, current_el, rot_tel=current_rot)
    

In [ ]:
# Now back the other way
current_az = 0.0
current_el = 80.0
current_rot = 160.0
await atcs.point_azel(current_az, current_el, rot_tel=current_rot)
for i in range(160):
    current_rot -= 2.0
    await latiss.take_bias(nbias=1)
    await atcs.point_azel(current_az, current_el, rot_tel=current_rot)
    

In [ ]:
# Now the same way, but with a delay
current_az = 0.0
current_el = 80.0
current_rot = -160.0
await atcs.point_azel(current_az, current_el, rot_tel=current_rot)
for i in range(160):
    current_rot += 2.0
    await latiss.take_bias(nbias=1)
    await atcs.point_azel(current_az, current_el, rot_tel=current_rot)
    await asyncio.sleep(5.0)

In [ ]:
current_az = 0.0
current_el = 80.0
current_rot = -112.0
await atcs.point_azel(current_az, current_el, rot_tel=current_rot)

In [ ]:
await latiss.take_bias(nbias=1)

In [ ]:
current_az = 0.0
current_el = 80.0
current_rot = 0.0
await atcs.point_azel(current_az, current_el, rot_tel=current_rot)

In [ ]:
await atcs.stop_tracking()

In [ ]:
await atcs.standby()
await latiss.standby()